# Total & Average Emissions
**Comparing the results of NEMED to AEMO CDEII Reporting**<br>
Using the package <code>nemed</code> to calculate daily total NEM emissions, and average emissions (or emissions intensity) over a historical period.

### Data Preparation
#### Import Packages

In [1]:
import nemed

# To generate plots shown
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# Open plot in browser (optional)
import plotly.io as pio
pio.renderers.default = "browser"

#### Define Parameters

In [3]:
cache = "E:/TEMPCACHE/" # Set this to an existing empty folder or leave as ""
                        # to create a new cache folder
start_time = "2019/07/01 00:00:00" # Define the start of the historical period.
end_time = "2020/07/01 00:00:00" # Define the end of the historical period.
filter_units = None # List DUIDs if you only require data for specific units.
filter_regions = None # List regions if you only require data for such.

#### Get data using NEMED

In [4]:
os.chdir(r'E:\PROJECTS\nemed-fork\NEMED\docs\source\examples\CACHE')
energy = pd.read_parquet('average_energy.parquet')
emissions_tot = pd.read_parquet('average_emissions.parquet')
emissions_avg = pd.read_parquet('average_intensity.parquet')
marginal = pd.read_parquet('marginal_emissions.parquet')

In [4]:
avg_em = nemed.get_total_emissions(start_time, end_time, cache, filter_regions,
                                   by="day", generation_sent_out=True, assume_ramp=True)

Compiling data for table DISPATCH_UNIT_SCADA.
Returning DISPATCH_UNIT_SCADA.
Retrieving static table Generators and Scheduled Loads.
Empty DataFrame
Columns: [Time, DUID, Dispatch]
Index: []


In [5]:
mar_em = nemed.get_marginal_emissions(start_time, end_time, cache, redownload_xml=True)

Retrieving static table Generators and Scheduled Loads.
Redownloading XML data
Converting selected 105941 XML files to JSON, of 117749 cached files


100%|██████████| 105941/105941 [20:19<00:00, 86.90it/s] 


Reading JSON to pandas Dataframe
Reading selected 105941 JSON files to pandas, of 117749 cached files


100%|██████████| 105941/105941 [06:05<00:00, 289.96it/s]


In [13]:
os.chdir(r'E:\PROJECTS\nemed-fork\NEMED\docs\source\examples\CACHE')
avg_em['Energy'].to_parquet('average_energy.parquet')
avg_em['Total_Emissions'].to_parquet('average_emissions.parquet')
avg_em['Intensity_Index'].to_parquet('average_intensity.parquet')
mar_em.to_parquet('marginal_emissions.parquet')

##### Sent Out Energy Generation

In [15]:
result = avg_em
energy = result['Energy']
energy

REGIONID,NSW1,QLD1,SA1,TAS1,VIC1
Time,,,,,
2019-07-01,188475.94,140237.70,42781.74,34422.89,128347.96
2019-07-02,191894.52,148292.08,30292.71,37466.29,128037.03
2019-07-03,195915.71,147842.50,32382.99,36772.20,120738.97
2019-07-04,190172.49,146418.21,40759.82,34460.97,119740.63
2019-07-05,184955.61,148981.39,42738.62,33594.55,117475.22
...,...,...,...,...,...
2020-06-26,187235.77,151978.74,26540.36,42658.39,124699.02
2020-06-27,177392.39,140081.19,27082.23,38169.58,115306.83
2020-06-28,172693.11,144011.89,28869.30,38397.33,115175.96


##### Total Emissions

In [16]:
emissions_tot = result['Total_Emissions']
emissions_tot

REGIONID,NSW1,QLD1,SA1,TAS1,VIC1
Time,,,,,
2019-07-01,156717.63,111057.75,7565.50,210.41,111884.39
2019-07-02,157130.76,118896.81,11346.85,566.65,120742.80
2019-07-03,158038.41,118374.61,12280.44,728.65,123418.79
2019-07-04,150211.49,118749.86,9343.21,216.28,117846.72
2019-07-05,142058.27,120173.60,8905.54,367.48,116196.27
...,...,...,...,...,...
2020-06-26,152962.99,112865.52,13843.18,238.55,125922.73
2020-06-27,145749.84,105427.07,11843.74,250.43,124437.84
2020-06-28,140498.46,109021.19,11481.43,128.74,124434.12


##### Emissions Intensity (Average Emissions)

In [17]:
emissions_avg = result['Intensity_Index']
emissions_avg

REGIONID,NSW1,QLD1,SA1,TAS1,VIC1
Time,,,,,
2019-07-01,0.84,0.79,0.17,0.01,0.87
2019-07-02,0.82,0.80,0.38,0.01,0.95
2019-07-03,0.81,0.80,0.38,0.02,1.03
2019-07-04,0.79,0.81,0.23,0.01,0.99
2019-07-05,0.77,0.81,0.21,0.01,0.99
...,...,...,...,...,...
2020-06-26,0.82,0.74,0.52,0.01,1.02
2020-06-27,0.82,0.75,0.43,0.01,1.09
2020-06-28,0.81,0.76,0.40,0.00,1.09


##### Total Metrics of NEM

In [11]:
energy['NEM'] = energy[['NSW1','QLD1','VIC1','SA1','TAS1']].sum(axis=1)
emissions_tot['NEM'] = emissions_tot[['NSW1','QLD1','VIC1','SA1','TAS1']].sum(axis=1)
emissions_avg['NEM'] = emissions_tot['NEM'] / energy['NEM']

#### Retrieve Published AEMO CDEII data for comparison

In [5]:
from nemed.downloader import download_aemo_cdeii_summary
aemo = download_aemo_cdeii_summary(start_time, end_time, cache=cache)
aemo

Extracting AEMO CDEII Datafile for: 2019, 2019
Extracting AEMO CDEII Datafile for: 2020, 2020


,SETTLEMENTDATE,REGIONID,TOTAL_SENT_OUT_ENERGY,TOTAL_EMISSIONS,CO2E_INTENSITY_INDEX
0,2019-07-01,NEM,560128.195652,402010.790601,0.7177
1,2019-07-01,NSW1,199074.132244,165700.401132,0.8324
2,2019-07-01,QLD1,149565.110362,114551.520028,0.7659
3,2019-07-01,SA1,44759.331498,8700.499581,0.1944
4,2019-07-01,TAS1,34072.798058,207.862436,0.0061
...,...,...,...,...,...
2197,2020-07-01,QLD1,153813.538903,112565.988220,0.7318
2198,2020-07-01,TAS1,30547.279940,1.640431,0.0001
2199,2020-07-01,NEM,542177.065812,375376.671409,0.6924
2200,2020-07-01,NSW1,184516.710491,144185.682308,0.7814


#### Compute Error between Datasets
The following functions calculate the error for each dataset (energy, total emissions, emissions intensity). Note error metrics are shown as zero when there is a difference between AEMO and NEMED < 50MW, or if the AEMO value is < 100. This avoids having extreme values which may be misinterpreted due to the underlying values simply being extremely small.

In [6]:
def calculate_total_emissions_error(region):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)
    error_calc = pd.Series([emissions_tot[region].values[i] - \
        sel_aemo['TOTAL_EMISSIONS'][i] for i in range(min(len(emissions_tot[region]), \
        len(sel_aemo['TOTAL_EMISSIONS'])))])
    numer = np.where(sel_aemo['TOTAL_EMISSIONS'] < 100, 0, error_calc)
    error_pct = [(numer[i] / sel_aemo['TOTAL_EMISSIONS'][i]) * 100 \
        for i in range(len(error_calc))]
    return error_pct

def calculate_energy_error(region):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)
    error_calc_en = pd.Series([energy[region].values[i] - \
        sel_aemo['TOTAL_SENT_OUT_ENERGY'][i] for i in range(min(len(energy[region]), \
        len(sel_aemo['TOTAL_SENT_OUT_ENERGY'])))])
    numer = np.where(sel_aemo['TOTAL_SENT_OUT_ENERGY'] < 100, 0, error_calc_en)
    error_pct_en = [(numer[i] / sel_aemo['TOTAL_SENT_OUT_ENERGY'][i]) * 100 \
        for i in range(len(error_calc_en))]
    return error_pct_en

def calculate_intensity_error(region):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)
    error_calc = pd.Series([emissions_avg[region].values[i] - \
        sel_aemo['CO2E_INTENSITY_INDEX'][i] for i in \
        range(min(len(emissions_avg[region]), len(sel_aemo['CO2E_INTENSITY_INDEX'])))])
    #numer = np.where(error_calc < 50, 0, error_calc)
    numer = np.where(sel_aemo['CO2E_INTENSITY_INDEX'] < 0.1, 0, error_calc) #numer)
    error_pct = [(numer[i] / sel_aemo['CO2E_INTENSITY_INDEX'][i]) * 100 \
        for i in range(len(error_calc))]
    return error_pct

#### Setup Plotting
##### Energy Plot Format

In [7]:
def plot_energy(region, color_idx):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)

    error_pct_en = calculate_energy_error(region)

    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05,
        specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]])
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Sent Out Generation<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Total Energy<br>(MWh)", row=1, col=1)
    fig.update_yaxes(title_text="Error wrt.<br>AEMO (%)", row=4, col=1)
    fig.update_xaxes(title_text="Date (Day)", row=4, col=1)

    fig.add_trace(go.Scatter(x=sel_aemo['SETTLEMENTDATE'],y=sel_aemo['TOTAL_SENT_OUT_ENERGY'],\
        name="AEMO CDEII Reported",mode="lines+markers",line_color=colors[7]),row=1,col=1)

    fig.add_trace(go.Scatter(x=energy.index,y=energy[region].values,\
        name="NEMED Calculated",mode="lines+markers",line_color=colors[color_idx]),row=1,col=1)

    fig.add_trace(go.Scatter(x=energy.index,y=error_pct_en,\
        name="Calculation Error", mode="lines",line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

##### Total Emissions Plot Format

In [8]:
def plot_emissions(region, color_idx):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)

    error_pct = calculate_total_emissions_error(region)
        
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
        specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]],)
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Total Emissions<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Total Emissions<br>(tCO2-e)", row=1, col=1)
    fig.update_yaxes(title_text="Error wrt.<br>AEMO (%)", row=4, col=1)
    fig.update_xaxes(title_text="Date (Day)", row=4, col=1)

    fig.add_trace(go.Scatter(x=sel_aemo['SETTLEMENTDATE'],y=sel_aemo['TOTAL_EMISSIONS'],\
        name="AEMO CDEII Report",mode="lines+markers",line_color=colors[7]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_tot.index,y=emissions_tot[region].values,\
        name="NEMED Calculated",mode="lines+markers",line_color=colors[color_idx]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_tot.index[:-1],y=error_pct,\
        name="Calculation Error", mode="lines",line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

##### Emissions Intensity (Average Emissions) Plot Format

In [9]:
def plot_intensity(region, color_idx):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)

    error_pct = calculate_intensity_error(region)
        
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
        specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]],)
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Average Emissions (Emissions Intensity)<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Emissions Intensity<br>(tCO2-e/MWh)", row=1, col=1)
    fig.update_yaxes(title_text="Error wrt.<br>AEMO (%)", row=4, col=1)
    fig.update_xaxes(title_text="Date (Day)", row=4, col=1)

    fig.add_trace(go.Scatter(x=sel_aemo['SETTLEMENTDATE'],y=sel_aemo['CO2E_INTENSITY_INDEX'],\
        name="AEMO CDEII Report",mode="lines+markers",line_color=colors[7]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_avg.index,y=emissions_avg[region].values,\
        name="NEMED Calculated",mode="lines+markers",line_color=colors[color_idx]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_avg.index[:-1],y=error_pct,\
        name="Calculation Error", mode="lines", line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

### Results - Comparison to AEMO

#### NEM

In [12]:
region = 'NEM'
plot_energy(region, color_idx=0)
plot_emissions(region, color_idx=0)
plot_intensity(region, color_idx=0)

ValueError: operands could not be broadcast together with shapes (367,) () (366,) 

```{include} charts_ex1/FY1920_nem_energy.html
:class: full-width
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_nem_energy.html)

```{include} charts_ex1/FY1920_nem_emissions.html
:class: full-width
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_nem_emissions.html)

```{include} charts_ex1/FY1920_nem_intensity.html
:class: full-width
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_nem_intensity.html)

#### NSW

In [13]:
region = 'NSW1'
plot_energy(region, color_idx=1)
plot_emissions(region, color_idx=1)
plot_intensity(region, color_idx=1)

ValueError: operands could not be broadcast together with shapes (367,) () (366,) 

```{include} charts_ex1/FY1920_vic_energy.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_nsw_energy.html)

```{include} charts_ex1/FY1920_nsw_emissions.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_nsw_emissions.html)

```{include} charts_ex1/FY1920_nsw_intensity.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_nsw_intensity.html)

#### VIC

In [20]:
region = 'VIC1'
plot_energy(region, color_idx=3)
plot_emissions(region, color_idx=3)
plot_intensity(region, color_idx=3)

```{include} charts_ex1/FY1920_vic_energy.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_vic_energy.html)

```{include} charts_ex1/FY1920_vic_emissions.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_vic_emissions.html)

```{include} charts_ex1/FY1920_vic_intensity.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_vic_intensity.html)

#### QLD

In [21]:
region = 'QLD1'
plot_energy(region, color_idx=2)
plot_emissions(region, color_idx=2)
plot_intensity(region, color_idx=2)

```{include} charts_ex1/FY1920_qld_energy.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_qld_energy.html)

```{include} charts_ex1/FY1920_qld_emissions.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_qld_emissions.html)

```{include} charts_ex1/FY1920_qld_intensity.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_qld_intensity.html)

#### SA

In [22]:
region = 'SA1'
plot_energy(region, color_idx=4)
plot_emissions(region, color_idx=4)
plot_intensity(region, color_idx=4)

```{include} charts_ex1/FY1920_sa_energy.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_sa_energy.html)

```{include} charts_ex1/FY1920_sa_emissions.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_sa_emissions.html)

```{include} charts_ex1/FY1920_sa_intensity.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_sa_intensity.html)

#### TAS

In [23]:
region = 'TAS1'
plot_energy(region, color_idx=5)
plot_emissions(region, color_idx=5)
plot_intensity(region, color_idx=5)

C:\Users\derlu\AppData\Local\Temp\ipykernel_25344\2425312539.py:28: RuntimeWarning:

invalid value encountered in double_scalars



```{include} charts_ex1/FY1920_tas_energy.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_tas_energy.html)

```{include} charts_ex1/FY1920_tas_emissions.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_tas_emissions.html)

```{include} charts_ex1/FY1920_tas_intensity.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/FY1920_tas_intensity.html)